In [20]:
start_date = '2016-01-01'
end_date = '2021-06-01'
target = "10d_ret_standscalar"
alpha_name = "AlphaNetV1_Ts_Standard"
alpha_list = [
# 'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
            'open_ts_norm_30d',
             'high_ts_norm_30d',
             'low_ts_norm_30d',
             'close_ts_norm_30d',
             'vwap_ts_norm_30d',
             'volume_ts_norm_30d',
             'pct_chg_ts_norm_30d',
             'turnover_ts_norm_30d',
             'free_turnover_ts_norm_30d'
             ]
shift = 5
sequence = 6
LR = [0.001,0.0001,0.00001]
epoch_num = [30,30,30]

In [11]:
feat_list = alpha_list.copy()
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
# alpha_list = [i +"_residual" for i in feat_list]

In [12]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

# config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
# print('Loading the configuration from ' + config_path)
# configs = namespace.load_namespace(config_path)
# FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
# alpha_list = list(FT.features_in_path.keys())[:5]

In [14]:
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
['COV_open_ts_norm_30d_high_ts_norm_30d_10', 'COV_open_ts_norm_30d_low_ts_norm_30d_10', 'COV_open_ts_norm_30d_close_ts_norm_30d_10', 'COV_open_ts_norm_30d_vwap_ts_norm_30d_10', 'COV_open_ts_norm_30d_volume_ts_norm_30d_10', 'COV_open_ts_norm_30d_pct_chg_ts_norm_30d_10', 'COV_open_ts_norm_30d_turnover_ts_norm_30d_10', 'COV_open_ts_norm_30d_free_turnover_ts_norm_30d_10', 'COV_high_ts_norm_30d_low_ts_norm_30d_10', 'COV_high_ts_norm_30d_close_ts_norm_30d_10', 'COV_high_ts_norm_30d_vwap_ts_norm_30d_10', 'COV_high_ts_norm_30d_volume_ts_norm_30d_10', 'COV_high_ts_norm_30d_pct_chg_ts_norm_30d_10', 'COV_high_ts_norm_30d_turnover_ts_norm_30d_10', 'COV_high_ts_norm_30d_free_turnover_ts_norm_30d_10', 'COV_low_ts_norm_30d_close_ts_norm_30d_10', 'COV_low_ts_norm_30d_vwap_ts_norm_30d_10', 'COV_low_ts_norm_30d_volume_ts_norm_30d_10', 'COV_low_ts_norm_30d_pct_chg_ts_norm_30d_10', 'COV_low_ts_norm_30d_turnove

100%|██████████| 1286/1286 [01:24<00:00, 15.24it/s]


COV_open_ts_norm_30d_high_ts_norm_30d_10  \
timestamp           ticker                                             
2016-02-23 09:30:00 002226                                  2.118626   
                    000012                                  1.385385   
                    300006                                  1.316347   
                    600680                                  1.256579   
                    002138                                  1.097194   
...                                                              ...   
2021-05-31 09:30:00 300717                                 -0.095210   
                    300196                                 -0.125749   
                    002811                                 -0.183855   
                    600697                                 -0.220197   
                    300554                                 -0.354210   

                            COV_open_ts_norm_30d_low_ts_norm_30d_10  \
timestamp           ticker                                            
2016-02-23 09:30:00 002226                                 3.098398   
                    000012                                 1.273276   
                    300006                                 1.311885   
                    600680                                 0.940093   
                    002138                                 0.869594   
...                                                             ...   
2021-05-31 09:30:00 300717                                 1.581677   
                    300196                                 0.139049   
                    002811                                 0.344324   
                    600697                                 1.568647   
                    300554                                 0.684618   

                            COV_open_ts_norm_30d_close_ts_norm_30d_10  \
timestamp           ticker                                              
2016-02-23 09:30:00 002226                                   1.965882   
                    000012                                   0.912091   
                    300006                                   0.872523   
                    600680                                   1.167624   
                    002138                                   0.765259   
...                                                               ...   
2021-05-31 09:30:00 300717                                   0.521486   
                    300196                                   0.037841   
                    002811                                  -0.034275   
                    600697                                   0.034206   
                    300554                                  -0.447443   

                            COV_open_ts_norm_30d_vwap_ts_norm_30d_10  \
timestamp           ticker                                             
2016-02-23 09:30:00 002226                                  2.275510   
                    000012                                  1.211576   
                    300006                                  1.084910   
                    600680                                  1.144667   
                    002138                                  1.107383   
...                                                              ...   
2021-05-31 09:30:00 300717                                  0.671533   
                    300196                                  0.032277   
                    002811                                  0.023595   
                    600697                                  0.201960   
                    300554                                 -0.248939   

                            COV_open_ts_norm_30d_volume_ts_norm_30d_10  \
timestamp           ticker                                               
2016-02-23 09:30:00 002226                                   -1.052403   
                    000012                                

# Shift

In [21]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:31<00:00, 53.64it/s] 


[25, 20, 15, 10, 5, 0]


100%|██████████| 1251/1251 [07:27<00:00,  2.80it/s]


COV_open_ts_norm_30d_high_ts_norm_30d_10_Shift_25  \
timestamp           ticker                                                      
2016-04-11 09:30:00 000001                                           0.190531   
                    000005                                           0.530672   
                    000006                                           0.799797   
                    000008                                           0.402912   
                    000009                                           0.626908   
...                                                                       ...   
2021-05-31 09:30:00 688679                                           0.513448   
                    688698                                           0.174227   
                    688777                                           0.201978   
                    688788                                           1.333835   
                    688981                                           0.129498   

                            COV_open_ts_norm_30d_low_ts_norm_30d_10_Shift_25  \
timestamp           ticker                                                     
2016-04-11 09:30:00 000001                                          0.226561   
                    000005                                          0.389043   
                    000006                                          0.789358   
                    000008                                          0.649853   
                    000009                                          0.538724   
...                                                                      ...   
2021-05-31 09:30:00 688679                                          0.541416   
                    688698                                          0.284807   
                    688777                                          0.139831   
                    688788                                          1.298586   
                    688981                                          0.142511   

                            COV_open_ts_norm_30d_close_ts_norm_30d_10_Shift_25  \
timestamp           ticker                                                       
2016-04-11 09:30:00 000001                                           0.160096    
                    000005                                           0.332137    
                    000006                                           0.581171    
                    000008                                           0.345375    
                    000009                                           0.389420    
...                                                                       ...    
2021-05-31 09:30:00 688679                                           0.368268    
                    688698                                           0.031452    
                    688777                                           0.152468    
                    688788                                           1.222605    
                    688981                                           0.078740    

                            COV_open_ts_norm_30d_vwap_ts_norm_30d_10_Shift_25  \
timestamp           ticker                                                      
2016-04-11 09:30:00 000001                                           0.214607   
                    000005                                           0.462765   
                    000006                                           0.752605   
                    000008                                           0.519006   
                    000009                                           0.608402   
...                                                                       ...   
2021-05-31 09:30:00 688679                                           0.515969   
                    688698                                           0.180678   
                    688777                              

# Task Generation

In [22]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
75,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
76,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
77,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


,start_date,end_date
0,2019-01-01,2019-06-01
1,2019-06-01,2020-01-01
2,2020-01-01,2020-06-01
3,2020-06-01,2021-01-01
4,2021-01-01,2021-06-01


In [23]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i" % (shift,sequence),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
0,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
1,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
2,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
3,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
4,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [24]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task[-20:]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
65,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
66,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
67,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
68,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
69,Original_Input_Residual_Shift_1_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",9,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
70,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
71,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
72,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
73,AlphaNetV1_Ts_Standard_Shift_10_Sequence_3,3,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [25]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)